In [10]:
import google.generativeai as genai
import configparser
import json
import re

# Load API key from creds.ini
def load_api_key(file_path="creds.ini"):
    config = configparser.ConfigParser()
    config.read(file_path)
    try:
        return config["google"]["api_key"]
    except KeyError:
        print("API key not found in creds.ini. Please add it in the google section")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Generate question data
def generate_question_data(model):
    """Generates a question, options, correct answer, and additional fact using an LLM."""
    try:
        prompt = f"""Generate a question about a common backyard animal such as a squirrel, raccoon, possum, bluejay, or cardinal.
        Provide 3 multiple choice options, and indicate the single correct answer. The format should be a JSON object with keys for "question", "options" (which is an array), "correct_answer", and "additional_fact". Do not include a title.

        Here is an example of the format you should use:
        {{
            "question": "What do I do with the acorns that I bury?",
            "options": ["Leave them be", "Remember their location for later", "Forget where I buried them"],
            "correct_answer": "Forget where I buried them",
            "additional_fact": "That helps plant thousands of trees!"
        }}
        """
        response = model.generate_content(prompt)
        if not response or not response.text:
            print("LLM returned an empty response.")
            return None

        question_data = response.text.strip()
        
        # Remove markdown code block if present
        match = re.match(r'```(?:json)?\s*(.*?)\s*```', question_data, re.DOTALL)
        if match:
             question_data = match.group(1).strip()
        
        try:
          return json.loads(question_data)
        except json.JSONDecodeError as e:
           print(f"JSON decode error: {e}")
           print(f"LLM output: {question_data}")
           return None

    except Exception as e:
       print(f"Error during LLM call: {e}")
       return None


# Load the API key
api_key = load_api_key()

if api_key:
    try:
        # Configure the Gemini API
        genai.configure(api_key=api_key)

        # Initialize a generative model
        model = genai.GenerativeModel("gemini-1.5-flash")

        # Call the generate_question_data function
        data = generate_question_data(model)
        
        if data:
          print("Data generated successfully:")
          print(json.dumps(data, indent=4))
        else:
          print("Failed to generate data.")

    except Exception as e:
         print(f"An error occurred during API call: {e}")

else:
    print("API key not loaded. Please check your creds.ini file")

Data generated successfully:
{
    "question": "Which of these is a common way a squirrel communicates with other squirrels?",
    "options": [
        "Singing complex melodies",
        "Using a variety of distinct barks and chatters",
        "Dancing elaborate mating rituals"
    ],
    "correct_answer": "Using a variety of distinct barks and chatters",
    "additional_fact": "Squirrels use a wide range of vocalizations, including chirps, chatters, and barks, to communicate alarm, location of food, and territorial boundaries."
}
